# LEAM TCN MNIST

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import numpy as np
import pickle

import sklearn.metrics as metrics
import os
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from scipy import spatial
import sklearn.metrics as Metrics

import itertools
from collections import Counter
import time
import seaborn as sns

import matplotlib.pyplot as plt


In [3]:
class Options(object):
    def __init__(self):
        # GPU id
        self.gpu_id = 0
        ###################### Data ##########################
        # Random state of spliting data
        self.rs = None
        self.num_input = None
        self.num_class = None
        self.timestep = None

        ###################### Model #########################
        # Training Batch Size
        self.batch_size = 20
        # Epoch
        self.epoch = 251
        # Learning rate
        self.lr_rate = 0.001
        # keep_prob, dropout_rate = 1 - keep_prob, here is the keep_prob rate
        self.keep_prob = 0.8
        # hidden units for notes
        self.H_dis = 4
        # Optimizer
        self.optimizer = 'Adam'
        # Validation Frequency
        self.valid_freq = 100
        # Early Stopping
        self.early_stop = False
        # Patience
        self.patience = None
        # Encoder
        self.encoder = "None"
        # Dilation rate
        self.l = 1
        # kernel size for tcn
        self.k = 3
        # number of filters
        self.num_filters = 8
        # save model path
        self.save_path = './save/leam_att/att_'

## MNIST Dataset

In [4]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("./data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Model

## Original TCN Model

In [51]:
import warnings; warnings.simplefilter('ignore')

In [87]:
class CausalConv1D(tf.layers.Conv1D):
    def __init__(self, filters,
               kernel_size,
               strides=1,
               dilation_rate=1,
               activation=None,
               use_bias=True,
               kernel_initializer=None,
               bias_initializer=tf.zeros_initializer(),
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               trainable=True,
               name=None,
               **kwargs):
        super(CausalConv1D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, **kwargs
        )
        
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1) * self.dilation_rate[0]
        if self.data_format == 'channels_first':
            inputs = tf.pad(inputs, tf.constant([[0, 0], [0, 0], [padding, 0]], dtype=tf.int32))
        else:
            inputs = tf.pad(inputs, tf.constant([(0, 0,), (padding, 0), (0, 0)]))
        return super(CausalConv1D, self).call(inputs), inputs

In [8]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.expand_dims(
        tf.expand_dims(tf.constant([1, 0, 0, 1, 0, 0, 1], dtype=tf.float32), axis=0),
        axis=-1) # (batch_size, length, channel)
    with tf.variable_scope("tcn"):
        conv = CausalConv1D(8, 2, dilation_rate=2, activation=None)
    output = conv(x)
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res, inputs = sess.run(output)
    print(inputs.shape)
    print(inputs[0, :, 0])
    print(res.shape)    
    print(res[0, :, 0])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(1, 9, 1)
[0. 0. 1. 0. 0. 1. 0. 0. 1.]
(1, 7, 8)
[-0.14872932  0.         -0.46489942 -0.14872932  0.         -0.46489942
 -0.14872932]


In [9]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 4, 10)) # (batch_size, channel, length)
    dropout = tf.layers.Dropout(0.5, noise_shape=[x.shape[0], x.shape[1], tf.constant(1)])
    output = dropout(x, training=True)
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, :])
    print(res[1, :, :])

(32, 4, 10)
[[ 0.0000000e+00  0.0000000e+00 -0.0000000e+00  0.0000000e+00
  -0.0000000e+00 -0.0000000e+00 -0.0000000e+00  0.0000000e+00
  -0.0000000e+00 -0.0000000e+00]
 [-0.0000000e+00 -0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
   0.0000000e+00 -0.0000000e+00]
 [ 6.7740923e-01  1.9230318e+00 -1.7757033e+00  2.2743883e+00
  -1.1445057e+00  4.4063632e-03  1.6304694e+00 -7.6573098e-01
   2.0447132e-01 -1.7233037e+00]
 [-1.7778189e-01  3.2459620e-01  3.3309302e+00  1.6665771e+00
   5.0001545e+00  2.5287453e-02  3.6113930e+00  3.0575074e-02
   1.1064229e+00 -1.7591003e+00]]
[[ 1.1488044  -0.20871945  0.13462071 -1.2171024  -1.6498863   2.488263
  -0.992963    1.4148185  -0.73274577 -3.3760247 ]
 [ 1.3042604   2.2655525  -2.0222583  -2.3801963  -1.686778    0.6138112
   2.658316   -1.1744169   2.2630975  -1.5531353 ]
 [-0.60847014  2.5457692   2.0453658   1.4912694   3.4782932   5.6132708
   0.35136226  1.7942569   1.2821397   

In [144]:
class TemporalBlock(tf.layers.Layer):
    def __init__(self, n_outputs, kernel_size, strides, dilation_rate, dropout=0.2, 
                 trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalBlock, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )        
        self.dropout = dropout
        self.n_outputs = n_outputs
        self.conv1 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv1")
        self.conv2 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv2")
        self.down_sample = None

    
    def build(self, input_shape):
        channel_dim = 2
        self.dropout1 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        self.dropout2 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        if input_shape[channel_dim] != self.n_outputs:
#             pass
            # self.down_sample = tf.layers.Conv1D(
            #     self.n_outputs, kernel_size=1, 
            #     activation=None, data_format="channels_last", padding="valid")
            self.down_sample = tf.layers.Dense(self.n_outputs, activation=None)
        self.built = True
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)[0]
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)[0]
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout2(x, training=training)
        if self.down_sample is not None:
            print('downsampling...')
            inputs = self.down_sample(inputs)
        return tf.nn.relu(x + inputs)

In [142]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 10, 4)) # (batch_size, length, channel)
    tblock = TemporalBlock(8, 2, 1, 1)
    output = tblock(x, training=tf.constant(True))
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, 0])
    print(res[1, :, 1])

(32, 10, 8)
[0.        0.        0.        0.        0.        0.        0.
 1.2673484 0.        0.       ]
[0.         0.43734848 4.0474186  2.0499163  2.4249654  1.4971675
 0.         0.         0.         0.8383116 ]


In [157]:
class TemporalConvNet(tf.layers.Layer):
    def __init__(self, num_channels, kernel_size=2, dropout=0.2,
                 trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalConvNet, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )
        self.layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            out_channels = num_channels[i]
            self.layers.append(
                TemporalBlock(out_channels, kernel_size, strides=1, dilation_rate=dilation_size,
                              dropout=dropout, name="tblock_{}".format(i))
            )
    
    def call(self, inputs, training=True):
        outputs = inputs
        for layer in self.layers:
            outputs = layer(outputs, training=training)
        return outputs

In [149]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 10, 4)) # (batch_size, length, channel)
    tcn = TemporalConvNet([8, 8, 8, 8], 2, 0.25)
    output = tcn(x, training=tf.constant(True))
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, 0])
    print(res[1, :, 1])

Number of levels 4
downsampling...
(32, 10, 8)
[0.         1.1321595  0.         0.68209094 0.         0.7974938
 0.         0.         2.4094856  2.0317686 ]
[2.1969428  6.531462   6.1554384  1.487592   0.         0.26302177
 0.93529415 0.         0.         0.        ]


In [173]:
def emb_classifier(x, y, dropout, opt, is_training):
    num_blocks = 6
    tblock = x
    layers = []
    for i in range(num_blocks):
        opt.l = 2 ** i
        tblock = temporal_block(tblock, dropout, opt, is_training)
#     # first block
#     opt.l = 1
#     layer_1 = temporal_block(x, dropout, opt, is_training)
#     # second block
#     opt.l = 2
#     layer_2 = temporal_block(layer_1, dropout, opt, is_training)
#     # third block
#     opt.l = 4
#     layer_3 = temporal_block(layer_2, dropout, opt, is_training)
#     # fourth block
#     opt.l = 8
#     layer_4 = temporal_block(layer_3, dropout, opt, is_training)
#     # print(layer_3.shape)
#     # print(layer_3[:, -1, :].shape)
    H_enc_fin = tblock[:, -1, :]

    logits = tf.layers.dense(H_enc_fin, 10, activation=None, kernel_initializer=tf.orthogonal_initializer())
#     logits = tf.layers.dense(
#         TemporalConvNet([20] * 6, opt.num_filters, 1-dropout)(
#             X, training=is_training)[:, -1, :],
#         opt.num_classes, activation=None, 
#         kernel_initializer=tf.orthogonal_initializer()
#     )
    prob = tf.nn.softmax(logits)
#     print(H_enc_fin.shape, logits.shape, y.shape)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prob, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    saver = tf.train.Saver()

    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_step = tf.train.AdamOptimizer(opt.lr_rate).minimize(loss)
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))
    return prob, loss, train_step, accuracy, saver, init

## Training

In [159]:
opt = Options()

In [175]:
opt.l = 1
opt.k = 20

opt.num_input = 1 # MNIST data input (img shape: 28*28)
opt.timestep = 28 * 28 # timesteps
opt.num_classes = 10 # MNIST total classes (0-9 digits)
opt.keep_prob = 0.9
opt.num_filters = 20

# Training Parameters
opt.lr_rate = 0.001
opt.batch_size = 64
display_step = 500
total_batch = int(mnist.train.num_examples / opt.batch_size)
print("Number of batches per epoch:", total_batch)
training_steps = 3000

Number of batches per epoch: 859


In [176]:
def temporal_block(x, dropout, opt, is_training):
    print("---- dialation {0} ----".format(opt.l))
    padding = (opt.k - 1) * opt.l
    # masked note embeddings
    # x_masked_notes = tf.multiply(x, x_mask_notes)
    x_masked_notes = x
    tcn_1 = CausalConv1D(opt.num_filters, opt.k, dilation_rate=opt.l, activation=None)(x)[0]
#     print(tcn_1[0].shape, tcn_1[1].shape)
#     x_padded = tf.pad(x_masked_notes, tf.constant([(0, 0), (padding, 0), (0, 0)]))
    # 1st tcn layer with dialation rate l and kernel size k
#     tcn_1 = tf.layers.conv1d(x_padded, filters=opt.num_filters, kernel_size=opt.k, padding='valid',
#                              dilation_rate=opt.l, activation=None)
    tcn_1_norm = tf.nn.relu(tf.contrib.layers.layer_norm(tcn_1))
    tcn_1_output = tf.layers.dropout(tcn_1_norm, rate=dropout, training=is_training, 
                                     noise_shape = [tf.constant(1), tf.constant(1), tf.constant(opt.num_filters)])
    # print(tcn_1_output.shape)
    # 2nd tcn layer with same specs
    # tcn_1_output_masked = tf.multiply(tcn_1_output, x_mask_notes)
    tcn_1_output_masked = tcn_1_output
#     x_padded_2 = tf.pad(tcn_1_output_masked, tf.constant([(0, 0), (padding, 0), (0, 0)]))
#     tcn_2 = tf.layers.conv1d(x_padded_2, filters=opt.num_filters, kernel_size=opt.k, padding='valid',
#                              dilation_rate=opt.l, activation=None)
    tcn_2 = CausalConv1D(opt.num_filters, opt.k, dilation_rate=opt.l, activation=None)(tcn_1_output_masked)[0]
    tcn_2_norm = tf.nn.relu(tf.contrib.layers.layer_norm(tcn_2))
    tcn_2_output = tf.layers.dropout(tcn_2_norm, rate=dropout, training=is_training,
                                    noise_shape = [tf.constant(1), tf.constant(1), tf.constant(opt.num_filters)])
    print(tcn_2_output.shape)
    return tcn_2_output

In [177]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    # tf Graph input
    X = tf.placeholder("float", [None, opt.timestep, opt.num_input], name='X')
    Y = tf.placeholder("float", [None, opt.num_classes], name='y')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob_')
    is_training = tf.placeholder("bool", name="trainable_")
    prob_, loss_, train_step_, acc_, saver_, init = emb_classifier(X, Y, keep_prob, opt, is_training)

---- dialation 1 ----
(?, 784, 20)
---- dialation 2 ----
(?, 784, 20)
---- dialation 4 ----
(?, 784, 20)
---- dialation 8 ----
(?, 784, 20)
---- dialation 16 ----
(?, 784, 20)
---- dialation 32 ----
(?, 784, 20)
All parameters: 267992.0
Trainable parameters: 89330


In [170]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8
with tf.Session(graph=graph, config=config) as sess:
    # Run the initializer
    sess.run(init)
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(opt.batch_size)
#         print(np.max(batch_x), np.mean(batch_x), np.median(batch_x))
        # Reshape data to get 28 * 28 seq of 1 elements
        batch_x = batch_x.reshape((opt.batch_size, opt.timestep, opt.num_input))
        # Run optimization op (backprop)
        loss, _ = sess.run([loss_, train_step_], feed_dict={X: batch_x, Y: batch_y, keep_prob: opt.keep_prob, is_training: True})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_, acc_], feed_dict={
                X: batch_x, Y: batch_y, keep_prob:1.0, is_training: False})
            # Calculate accuracy for 128 mnist test images
            test_len = 128
            test_data = mnist.test.images[:test_len].reshape((-1, opt.timestep, opt.num_input))
            test_label = mnist.test.labels[:test_len]
            val_acc = sess.run(acc_, feed_dict={X: test_data, Y: test_label, keep_prob:1.0,is_training: False})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Test Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver_.save(sess, "./save/model.ckpt")
                print("Model saved in path: %s" % save_path)
    print("Optimization Finished!")

Step 1, Minibatch Loss= 3.9256, Training Accuracy= 0.109, Test Accuracy= 0.086
Step 500, Minibatch Loss= 2.3747, Training Accuracy= 0.016, Test Accuracy= 0.078
Step 1000, Minibatch Loss= 2.3196, Training Accuracy= 0.078, Test Accuracy= 0.078
Step 1500, Minibatch Loss= 2.3290, Training Accuracy= 0.047, Test Accuracy= 0.078
Step 2000, Minibatch Loss= 2.2950, Training Accuracy= 0.125, Test Accuracy= 0.078
Step 2500, Minibatch Loss= 2.2945, Training Accuracy= 0.125, Test Accuracy= 0.094
Step 3000, Minibatch Loss= 2.3012, Training Accuracy= 0.125, Test Accuracy= 0.094
Optimization Finished!
